In [ ]:
import urllib.request, json
import xmltodict
import pandas as pd
with urllib.request.urlopen("https://www.fire.nsw.gov.au/feeds/stations-georss.php") as url:
    # print(url.read())
    o = xmltodict.parse(url.read().decode())
    strjsondata = json.dumps(o)
    jsondata = json.loads(strjsondata)
    df = pd.DataFrame.from_dict(jsondata['rss']['channel']['item'], orient="columns")
    df.to_csv("fire_station_NSW.csv")

In [52]:
import geopandas as gpd
import requests
from zipfile import ZipFile
import io
import pandas as pd

In [53]:
fs = pd.read_csv("./dataset/fire_stations/NSW_fire_station.csv")
fs.head()

,Unnamed: 0,pubDate,title,description,link,geo:lat,geo:long
0,0,"Sat, 04 Nov 2023 09:45:11 +1100","208, Aberdeen Fire Station","20 Moray St, Aberdeen NSW 2336",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.163444,150.890079
1,1,"Sat, 04 Nov 2023 09:45:11 +1100","202, Abermain Fire Station","Cnr Cessnock Road & Charles Street, Abermain N...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.811131,151.431850
2,2,"Sat, 04 Nov 2023 09:45:11 +1100","207, Albion Park Fire Station","3 Russell Street, Albion Park NSW 2527",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-34.572809,150.774547
3,3,"Sat, 04 Nov 2023 09:45:11 +1100","203, Albury Central Fire Station","817 - 821 Mate Street, Albury Central (Albury ...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.065825,146.930701
4,4,"Sat, 04 Nov 2023 09:45:11 +1100","209, Albury Civic Fire Station","565 Kiewa Street, Albury Civic NSW 2640",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.077125,146.915934


In [56]:
from shapely import Point
from ast import literal_eval

In [55]:
fs_new = fs.drop('Unnamed: 0', axis=1)
fs_new.hea

,pubDate,title,description,link,geo:lat,geo:long
0,"Sat, 04 Nov 2023 09:45:11 +1100","208, Aberdeen Fire Station","20 Moray St, Aberdeen NSW 2336",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.163444,150.890079
1,"Sat, 04 Nov 2023 09:45:11 +1100","202, Abermain Fire Station","Cnr Cessnock Road & Charles Street, Abermain N...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.811131,151.431850
2,"Sat, 04 Nov 2023 09:45:11 +1100","207, Albion Park Fire Station","3 Russell Street, Albion Park NSW 2527",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-34.572809,150.774547
3,"Sat, 04 Nov 2023 09:45:11 +1100","203, Albury Central Fire Station","817 - 821 Mate Street, Albury Central (Albury ...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.065825,146.930701
4,"Sat, 04 Nov 2023 09:45:11 +1100","209, Albury Civic Fire Station","565 Kiewa Street, Albury Civic NSW 2640",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.077125,146.915934


In [58]:
fs_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   pubDate      335 non-null    object 
 1   title        335 non-null    object 
 2   description  335 non-null    object 
 3   link         335 non-null    object 
 4   geo:lat      335 non-null    float64
 5   geo:long     335 non-null    float64
dtypes: float64(2), object(4)
memory usage: 15.8+ KB


In [64]:
fs_new['coord'] = fs_new.apply(lambda x: Point(x['geo:long'], x['geo:lat']), axis=1)
fs_new.head()

,pubDate,title,description,link,geo:lat,geo:long,coord
0,"Sat, 04 Nov 2023 09:45:11 +1100","208, Aberdeen Fire Station","20 Moray St, Aberdeen NSW 2336",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.163444,150.890079,POINT (150.890079003099 -32.1634444097505)
1,"Sat, 04 Nov 2023 09:45:11 +1100","202, Abermain Fire Station","Cnr Cessnock Road & Charles Street, Abermain N...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.811131,151.431850,POINT (151.43185 -32.811131)
2,"Sat, 04 Nov 2023 09:45:11 +1100","207, Albion Park Fire Station","3 Russell Street, Albion Park NSW 2527",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-34.572809,150.774547,POINT (150.774547 -34.572809)
3,"Sat, 04 Nov 2023 09:45:11 +1100","203, Albury Central Fire Station","817 - 821 Mate Street, Albury Central (Albury ...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.065825,146.930701,POINT (146.930701 -36.065825)
4,"Sat, 04 Nov 2023 09:45:11 +1100","209, Albury Civic Fire Station","565 Kiewa Street, Albury Civic NSW 2640",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.077125,146.915934,POINT (146.915934 -36.077125)


In [67]:
fs_new_gdf = gpd.GeoDataFrame(fs_new, crs="EPSG:4376", geometry=[x for x in fs_new.coord])
fs_new_gdf.head()

,pubDate,title,description,link,geo:lat,geo:long,coord,geometry
0,"Sat, 04 Nov 2023 09:45:11 +1100","208, Aberdeen Fire Station","20 Moray St, Aberdeen NSW 2336",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.163444,150.890079,POINT (150.890079003099 -32.1634444097505),POINT (150.89008 -32.16344)
1,"Sat, 04 Nov 2023 09:45:11 +1100","202, Abermain Fire Station","Cnr Cessnock Road & Charles Street, Abermain N...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-32.811131,151.431850,POINT (151.43185 -32.811131),POINT (151.43185 -32.81113)
2,"Sat, 04 Nov 2023 09:45:11 +1100","207, Albion Park Fire Station","3 Russell Street, Albion Park NSW 2527",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-34.572809,150.774547,POINT (150.774547 -34.572809),POINT (150.77455 -34.57281)
3,"Sat, 04 Nov 2023 09:45:11 +1100","203, Albury Central Fire Station","817 - 821 Mate Street, Albury Central (Albury ...",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.065825,146.930701,POINT (146.930701 -36.065825),POINT (146.93070 -36.06582)
4,"Sat, 04 Nov 2023 09:45:11 +1100","209, Albury Civic Fire Station","565 Kiewa Street, Albury Civic NSW 2640",https://www.fire.nsw.gov.au/page.php?id=9210&s...,-36.077125,146.915934,POINT (146.915934 -36.077125),POINT (146.91593 -36.07713)


In [68]:
fs_new_gdf = fs_new_gdf.drop('coord', axis=1)
fs_new_gdf.to_file("./dataset/nsw_fs_dataset")

C:\Users\n9502271\AppData\Local\Temp\ipykernel_36728\1623182602.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fs_new_gdf.to_file("./dataset/nsw_fs_dataset")


In [2]:
local_path = "./dataset/"
wildfire = pd.read_csv(f"{local_path}wildfire_data.csv")
wildfire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14334 entries, 0 to 14333
Data columns (total 58 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Fire_ID              14334 non-null  int64  
 1   Date                 14334 non-null  int64  
 2   AreaHa               14334 non-null  float64
 3   PerimeterM           14334 non-null  float64
 4   air_mean             14334 non-null  float64
 5   air_min              14334 non-null  float64
 6   air_max              14334 non-null  float64
 7   pre_mean             14334 non-null  float64
 8   pre_min              14334 non-null  float64
 9   pre_max              14334 non-null  float64
 10  soilw_mean           14334 non-null  float64
 11  soilw_min            14334 non-null  float64
 12  soilw_max            14334 non-null  float64
 13  air_mean_spring      14334 non-null  float64
 14  air_mean_summer      14334 non-null  float64
 15  air_mean_autumn      14334 non-null 

In [3]:
from ast import literal_eval
from shapely import Point

In [4]:
wildfire["geometry"] = wildfire["geometry"].apply(lambda c: Point(literal_eval(", ".join(c.strip("POINT ()").split(" ")))))

In [5]:
wildfire.head()

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,soilw_max_winter,wind_mean (m.s++-1),wind_min (m.s++-1),wind_max (m.s++-1),Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_mean,Classify,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.593708,5,POINT (150.0861388164024 -34.24668639480927)
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0672479394609 -34.13110194566074)
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0737348331977 -34.13389252032015)
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.0760657232327 -34.14967444434212)
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.1532,Blue Mountains - South,Blue Mountains - South,0.769183,1,POINT (150.1331907652882 -34.22686843348224)


In [6]:
wildfire_gpd = gpd.GeoDataFrame(wildfire, crs="EPSG:4326", geometry = wildfire.geometry)

In [7]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json

import ipyleaflet
map = ipyleaflet.Map(center=(-27.47723161340788,153.02734507771476), zoom=15)
map


Map(center=[-27.47723161340788, 153.02734507771476], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [8]:
wildfire_gpd.head(10)

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,soilw_max_winter,wind_mean (m.s++-1),wind_min (m.s++-1),wind_max (m.s++-1),Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_mean,Classify,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.593708,5,POINT (150.08614 -34.24669)
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.06725 -34.13110)
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.07373 -34.13389)
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.948032,2.386662,3.670818,14.15320,Blue Mountains - South,Blue Mountains - South,0.790283,5,POINT (150.07607 -34.14967)
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.15320,Blue Mountains - South,Blue Mountains - South,0.769183,1,POINT (150.13319 -34.22687)
5,63,1982,4161.875464,43699.485003,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.765850,5,POINT (150.17446 -34.00031)
6,64,1982,1.310912,433.266553,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.804033,5,POINT (150.12770 -34.08072)
7,123,1982,574.156662,11278.583394,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.323638,1.916126,2.745639,6.56977,The Oaks - Oakdale,Wollondilly,0.717108,5,POINT (150.46133 -34.05408)
8,212,1982,2.058188,525.972898,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.653922,2.148278,3.265098,14.15320,Blue Mountains - South,Blue Mountains - South,0.813617,5,POINT (150.16990 -34.17413)
9,213,1982,4.345687,776.619843,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,321.04398,2.428833,1.999950,3.011134,14.15320,Blue Mountains - South,Blue Mountains - South,0.802233,5,POINT (150.19992 -34.11703)


In [9]:
geo_data = GeoData(geo_dataframe = wildfire_gpd.tail(30),
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

map.add_layer(geo_data)
map.add_control(LayersControl())

In [11]:
geo_gpd = gpd.read_file("./dataset/wildfire_sa2_level/wildfire_sa2_level.shp")
geo_gpd.head()

,Fire_ID,Date,AreaHa,PerimeterM,air_mean,air_min,air_max,pre_mean,pre_min,pre_max,...,wind_min (,wind_max (,Slope_mean,SA2_NAME16,SA3_NAME16,grid_code_,Classify,index_righ,SA2_CODE21,geometry
0,57,1982,1994.097836,22524.691340,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.593708,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
1,58,1982,1.362856,467.308070,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
2,59,1982,33.048317,3252.077734,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
3,61,1982,4.263715,1008.487720,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.386662,3.670818,14.1532,Blue Mountains - South,Blue Mountains - South,0.790283,5,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."
4,62,1982,1696.805552,25635.191330,14.211675,6.209992,21.160004,44.509094,4.4,127.6,...,2.148278,3.265098,14.1532,Blue Mountains - South,Blue Mountains - South,0.769183,1,526,124021456,"POLYGON ((150.17345 -33.81724, 150.17340 -33.8..."


In [12]:
import geopandas as gpd

In [7]:
nsw_fire = gpd.read_file(f"{local_path}NSW_fire_history_polygons.geojson")
nsw_fire.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry
0,1,,,1968-69 Prescribed Burn,1968-10-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,2.184770,1548.960666,26/03/2020,Prescribed Burn,1968-69,0,1968,1968-01-01 00:00:00+00:00,2.186876e+04,1549.703956,"POLYGON ((153.14143 -30.17730, 153.14144 -30.1..."
1,2,,,1970-71 Prescribed Burn,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,299.985464,8615.637120,26/03/2020,Prescribed Burn,1970-71,1,1970,1970-01-01 00:00:00+00:00,2.996520e+06,8610.851395,"POLYGON ((152.77796 -31.63806, 152.77822 -31.6..."
2,3,,70/161,1970-71 Prescribed Burn,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,309.025645,20657.633993,26/03/2020,Prescribed Burn,1970-71,2,1970,1970-01-01 00:00:00+00:00,3.085906e+06,20643.091296,"POLYGON ((151.50894 -31.98245, 151.50911 -31.9..."
3,4,,70/161,1970-71 Prescribed Burn,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,316.104427,20353.231751,26/03/2020,Prescribed Burn,1970-71,3,1970,1970-01-01 00:00:00+00:00,3.156677e+06,20339.165555,"POLYGON ((151.51755 -31.95905, 151.51787 -31.9..."
4,5,,70/160,1970-71 Prescribed Burn,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,28.484088,2695.160793,26/03/2020,Prescribed Burn,1970-71,4,1970,1970-01-01 00:00:00+00:00,2.844296e+05,2693.214793,"POLYGON ((151.60946 -32.03948, 151.60960 -32.0..."


In [8]:
nsw_csv = pd.read_csv(f"{local_path}NSW_fire_history_polygons.csv")
nsw_csv.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length
0,1,,,1968-69 Prescribed Burn,1968/10/01 00:00:00+00,1970/01/01 00:00:00+00,2.184770,1548.960666,26/03/2020,Prescribed Burn,1968-69,0,1968,1968/01/01 00:00:00+00,2.186876e+04,1549.703956
1,2,,,1970-71 Prescribed Burn,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,299.985464,8615.637120,26/03/2020,Prescribed Burn,1970-71,1,1970,1970/01/01 00:00:00+00,2.996520e+06,8610.851395
2,3,,70/161,1970-71 Prescribed Burn,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,309.025645,20657.633993,26/03/2020,Prescribed Burn,1970-71,2,1970,1970/01/01 00:00:00+00,3.085906e+06,20643.091296
3,4,,70/161,1970-71 Prescribed Burn,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,316.104427,20353.231751,26/03/2020,Prescribed Burn,1970-71,3,1970,1970/01/01 00:00:00+00,3.156677e+06,20339.165555
4,5,,70/160,1970-71 Prescribed Burn,1970/01/01 00:00:00+00,1970/01/01 00:00:00+00,28.484088,2695.160793,26/03/2020,Prescribed Burn,1970-71,4,1970,1970/01/01 00:00:00+00,2.844296e+05,2693.214793


In [10]:
nsw_fire.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33804 entries, 0 to 33803
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   FID            33804 non-null  int64              
 1   FireName       33804 non-null  object             
 2   FireNo         33804 non-null  object             
 3   Label          33804 non-null  object             
 4   StartDate      33804 non-null  datetime64[ns, UTC]
 5   EndDate        33804 non-null  datetime64[ns, UTC]
 6   AreaHa         33804 non-null  float64            
 7   PerimeterM     33804 non-null  float64            
 8   Verdate        33804 non-null  object             
 9   Type           33804 non-null  object             
 10  Season         33804 non-null  object             
 11  ID             33804 non-null  int64              
 12  Year           33804 non-null  int64              
 13  Year_Con       33804 non-null  datetim

In [38]:
nsw_fire_16_19 = nsw_fire[nsw_fire.Year.isin([2019])]

In [39]:
nsw_fire_16_19.shape

(542, 17)

In [40]:
nsw_fire_16_19.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [41]:
aus_url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
local_path = './dataset/'
print('Downloading shapefile...')
r = requests.get(aus_url)
z = ZipFile(io.BytesIO(r.content))
print("Done")
z.extractall(path=local_path) # extract to folder
filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)

Done
['SA2_2021_AUST_GDA2020.dbf', 'SA2_2021_AUST_GDA2020.prj', 'SA2_2021_AUST_GDA2020.shp', 'SA2_2021_AUST_GDA2020.shx']


In [42]:
dbf, prj, shp, shx = [filename for filename in filenames]
aus = gpd.read_file(local_path + shp)
print("Shape of the dataframe: {}".format(aus.shape))
print("Projection of dataframe: {}".format(aus.crs))
aus.tail() #last 5 records in dataframe

Shape of the dataframe: (2473, 17)
Projection of dataframe: EPSG:7844


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2472,ZZZZZZZZZ,Outside Australia,1,New,ZZZZZ,Outside Australia,ZZZ,Outside Australia,ZZZZZ,Outside Australia,Z,Outside Australia,ZZZ,Outside Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [43]:
aus_new = aus[~aus.isna()]
print("Shape of the dataframe: {}".format(aus_new.shape))
print("Projection of dataframe: {}".format(aus_new.crs))
aus_new_4326 = aus_new.to_crs("EPSG:4326")

Shape of the dataframe: (2473, 17)
Projection of dataframe: EPSG:7844


In [44]:
nsw_fire_new = nsw_fire_16_19.sjoin(aus_new_4326[['SA2_CODE21', 'geometry']])
nsw_fire_new.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry,index_right,SA2_CODE21
33250,33251,Peak Creek HR,HR16100778164,2019-20 Prescribed Burn,2019-07-16 00:00:00+00:00,2019-07-25 00:00:00+00:00,2469.017395,33000.948676,26/03/2020,Prescribed Burn,2019-20,33390,2019,2019-01-01 00:00:00+00:00,2.471877e+07,33020.026523,"POLYGON ((152.32185 -30.03437, 152.32304 -30.0...",82,104011081
33274,33275,Horseshoe HR,HR16091377943,2019-20 Prescribed Burn,2019-07-25 00:00:00+00:00,2019-07-31 00:00:00+00:00,504.410927,14519.597188,26/03/2020,Prescribed Burn,2019-20,33414,2019,2019-01-01 00:00:00+00:00,5.051272e+06,14529.917281,"POLYGON ((152.42029 -29.92220, 152.42055 -29.9...",82,104011081
33276,33277,Mann River South HR,HR17091281680,2019-20 Prescribed Burn,2019-07-19 00:00:00+00:00,2019-07-25 00:00:00+00:00,1486.149445,80869.621718,26/03/2020,Prescribed Burn,2019-20,33416,2019,2019-01-01 00:00:00+00:00,1.489017e+07,80947.604522,"POLYGON ((152.46986 -29.64997, 152.46842 -29.6...",82,104011081
33279,33280,"Second Creek, Washpool State Forest",19081744832,2019-20 Wildfire,2019-08-17 00:00:00+00:00,2019-08-30 00:00:00+00:00,703.944479,24752.031789,26/03/2020,Wildfire,2019-20,33419,2019,2019-01-01 00:00:00+00:00,7.058570e+06,24785.607812,"POLYGON ((152.39147 -29.32209, 152.39151 -29.3...",82,104011081
33299,33300,"Kingsgate, Red Range",19090948136,2019-20 Wildfire,2019-09-08 00:00:00+00:00,1969-12-31 00:00:00+00:00,35930.759165,265893.848750,26/03/2020,Wildfire,2019-20,33439,2019,2019-01-01 00:00:00+00:00,3.599022e+08,266106.773914,"POLYGON ((152.06953 -29.71374, 152.06959 -29.7...",82,104011081


In [45]:
nsw_fire_new.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 829 entries, 33250 to 33803
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   FID            829 non-null    int64              
 1   FireName       829 non-null    object             
 2   FireNo         829 non-null    object             
 3   Label          829 non-null    object             
 4   StartDate      829 non-null    datetime64[ns, UTC]
 5   EndDate        829 non-null    datetime64[ns, UTC]
 6   AreaHa         829 non-null    float64            
 7   PerimeterM     829 non-null    float64            
 8   Verdate        829 non-null    object             
 9   Type           829 non-null    object             
 10  Season         829 non-null    object             
 11  ID             829 non-null    int64              
 12  Year           829 non-null    int64              
 13  Year_Con       829 non-null    datetime64

In [46]:
nsw_fire_new.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry,index_right,SA2_CODE21
33250,33251,Peak Creek HR,HR16100778164,2019-20 Prescribed Burn,2019-07-16 00:00:00+00:00,2019-07-25 00:00:00+00:00,2469.017395,33000.948676,26/03/2020,Prescribed Burn,2019-20,33390,2019,2019-01-01 00:00:00+00:00,2.471877e+07,33020.026523,"POLYGON ((152.32185 -30.03437, 152.32304 -30.0...",82,104011081
33274,33275,Horseshoe HR,HR16091377943,2019-20 Prescribed Burn,2019-07-25 00:00:00+00:00,2019-07-31 00:00:00+00:00,504.410927,14519.597188,26/03/2020,Prescribed Burn,2019-20,33414,2019,2019-01-01 00:00:00+00:00,5.051272e+06,14529.917281,"POLYGON ((152.42029 -29.92220, 152.42055 -29.9...",82,104011081
33276,33277,Mann River South HR,HR17091281680,2019-20 Prescribed Burn,2019-07-19 00:00:00+00:00,2019-07-25 00:00:00+00:00,1486.149445,80869.621718,26/03/2020,Prescribed Burn,2019-20,33416,2019,2019-01-01 00:00:00+00:00,1.489017e+07,80947.604522,"POLYGON ((152.46986 -29.64997, 152.46842 -29.6...",82,104011081
33279,33280,"Second Creek, Washpool State Forest",19081744832,2019-20 Wildfire,2019-08-17 00:00:00+00:00,2019-08-30 00:00:00+00:00,703.944479,24752.031789,26/03/2020,Wildfire,2019-20,33419,2019,2019-01-01 00:00:00+00:00,7.058570e+06,24785.607812,"POLYGON ((152.39147 -29.32209, 152.39151 -29.3...",82,104011081
33299,33300,"Kingsgate, Red Range",19090948136,2019-20 Wildfire,2019-09-08 00:00:00+00:00,1969-12-31 00:00:00+00:00,35930.759165,265893.848750,26/03/2020,Wildfire,2019-20,33439,2019,2019-01-01 00:00:00+00:00,3.599022e+08,266106.773914,"POLYGON ((152.06953 -29.71374, 152.06959 -29.7...",82,104011081


In [47]:
nsw_fire_new['StartDate'] = nsw_fire_new['StartDate'].apply(lambda x: x.strftime("%m/%d/%Y, %H:%M:%S"))
nsw_fire_new.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry,index_right,SA2_CODE21
33250,33251,Peak Creek HR,HR16100778164,2019-20 Prescribed Burn,"07/16/2019, 00:00:00",2019-07-25 00:00:00+00:00,2469.017395,33000.948676,26/03/2020,Prescribed Burn,2019-20,33390,2019,2019-01-01 00:00:00+00:00,2.471877e+07,33020.026523,"POLYGON ((152.32185 -30.03437, 152.32304 -30.0...",82,104011081
33274,33275,Horseshoe HR,HR16091377943,2019-20 Prescribed Burn,"07/25/2019, 00:00:00",2019-07-31 00:00:00+00:00,504.410927,14519.597188,26/03/2020,Prescribed Burn,2019-20,33414,2019,2019-01-01 00:00:00+00:00,5.051272e+06,14529.917281,"POLYGON ((152.42029 -29.92220, 152.42055 -29.9...",82,104011081
33276,33277,Mann River South HR,HR17091281680,2019-20 Prescribed Burn,"07/19/2019, 00:00:00",2019-07-25 00:00:00+00:00,1486.149445,80869.621718,26/03/2020,Prescribed Burn,2019-20,33416,2019,2019-01-01 00:00:00+00:00,1.489017e+07,80947.604522,"POLYGON ((152.46986 -29.64997, 152.46842 -29.6...",82,104011081
33279,33280,"Second Creek, Washpool State Forest",19081744832,2019-20 Wildfire,"08/17/2019, 00:00:00",2019-08-30 00:00:00+00:00,703.944479,24752.031789,26/03/2020,Wildfire,2019-20,33419,2019,2019-01-01 00:00:00+00:00,7.058570e+06,24785.607812,"POLYGON ((152.39147 -29.32209, 152.39151 -29.3...",82,104011081
33299,33300,"Kingsgate, Red Range",19090948136,2019-20 Wildfire,"09/08/2019, 00:00:00",1969-12-31 00:00:00+00:00,35930.759165,265893.848750,26/03/2020,Wildfire,2019-20,33439,2019,2019-01-01 00:00:00+00:00,3.599022e+08,266106.773914,"POLYGON ((152.06953 -29.71374, 152.06959 -29.7...",82,104011081


In [48]:
nsw_fire_new['EndDate'] = nsw_fire_new['EndDate'].apply(lambda x: x.strftime("%m/%d/%Y, %H:%M:%S"))
nsw_fire_new.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry,index_right,SA2_CODE21
33250,33251,Peak Creek HR,HR16100778164,2019-20 Prescribed Burn,"07/16/2019, 00:00:00","07/25/2019, 00:00:00",2469.017395,33000.948676,26/03/2020,Prescribed Burn,2019-20,33390,2019,2019-01-01 00:00:00+00:00,2.471877e+07,33020.026523,"POLYGON ((152.32185 -30.03437, 152.32304 -30.0...",82,104011081
33274,33275,Horseshoe HR,HR16091377943,2019-20 Prescribed Burn,"07/25/2019, 00:00:00","07/31/2019, 00:00:00",504.410927,14519.597188,26/03/2020,Prescribed Burn,2019-20,33414,2019,2019-01-01 00:00:00+00:00,5.051272e+06,14529.917281,"POLYGON ((152.42029 -29.92220, 152.42055 -29.9...",82,104011081
33276,33277,Mann River South HR,HR17091281680,2019-20 Prescribed Burn,"07/19/2019, 00:00:00","07/25/2019, 00:00:00",1486.149445,80869.621718,26/03/2020,Prescribed Burn,2019-20,33416,2019,2019-01-01 00:00:00+00:00,1.489017e+07,80947.604522,"POLYGON ((152.46986 -29.64997, 152.46842 -29.6...",82,104011081
33279,33280,"Second Creek, Washpool State Forest",19081744832,2019-20 Wildfire,"08/17/2019, 00:00:00","08/30/2019, 00:00:00",703.944479,24752.031789,26/03/2020,Wildfire,2019-20,33419,2019,2019-01-01 00:00:00+00:00,7.058570e+06,24785.607812,"POLYGON ((152.39147 -29.32209, 152.39151 -29.3...",82,104011081
33299,33300,"Kingsgate, Red Range",19090948136,2019-20 Wildfire,"09/08/2019, 00:00:00","12/31/1969, 00:00:00",35930.759165,265893.848750,26/03/2020,Wildfire,2019-20,33439,2019,2019-01-01 00:00:00+00:00,3.599022e+08,266106.773914,"POLYGON ((152.06953 -29.71374, 152.06959 -29.7...",82,104011081


In [49]:
nsw_fire_new['Year_Con'] = nsw_fire_new['Year_Con'].apply(lambda x: x.strftime("%m/%d/%Y, %H:%M:%S"))
nsw_fire_new.head()

,FID,FireName,FireNo,Label,StartDate,EndDate,AreaHa,PerimeterM,Verdate,Type,Season,ID,Year,Year_Con,Shape__Area,Shape__Length,geometry,index_right,SA2_CODE21
33250,33251,Peak Creek HR,HR16100778164,2019-20 Prescribed Burn,"07/16/2019, 00:00:00","07/25/2019, 00:00:00",2469.017395,33000.948676,26/03/2020,Prescribed Burn,2019-20,33390,2019,"01/01/2019, 00:00:00",2.471877e+07,33020.026523,"POLYGON ((152.32185 -30.03437, 152.32304 -30.0...",82,104011081
33274,33275,Horseshoe HR,HR16091377943,2019-20 Prescribed Burn,"07/25/2019, 00:00:00","07/31/2019, 00:00:00",504.410927,14519.597188,26/03/2020,Prescribed Burn,2019-20,33414,2019,"01/01/2019, 00:00:00",5.051272e+06,14529.917281,"POLYGON ((152.42029 -29.92220, 152.42055 -29.9...",82,104011081
33276,33277,Mann River South HR,HR17091281680,2019-20 Prescribed Burn,"07/19/2019, 00:00:00","07/25/2019, 00:00:00",1486.149445,80869.621718,26/03/2020,Prescribed Burn,2019-20,33416,2019,"01/01/2019, 00:00:00",1.489017e+07,80947.604522,"POLYGON ((152.46986 -29.64997, 152.46842 -29.6...",82,104011081
33279,33280,"Second Creek, Washpool State Forest",19081744832,2019-20 Wildfire,"08/17/2019, 00:00:00","08/30/2019, 00:00:00",703.944479,24752.031789,26/03/2020,Wildfire,2019-20,33419,2019,"01/01/2019, 00:00:00",7.058570e+06,24785.607812,"POLYGON ((152.39147 -29.32209, 152.39151 -29.3...",82,104011081
33299,33300,"Kingsgate, Red Range",19090948136,2019-20 Wildfire,"09/08/2019, 00:00:00","12/31/1969, 00:00:00",35930.759165,265893.848750,26/03/2020,Wildfire,2019-20,33439,2019,"01/01/2019, 00:00:00",3.599022e+08,266106.773914,"POLYGON ((152.06953 -29.71374, 152.06959 -29.7...",82,104011081


In [50]:
nsw_fire_new.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 829 entries, 33250 to 33803
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   FID            829 non-null    int64   
 1   FireName       829 non-null    object  
 2   FireNo         829 non-null    object  
 3   Label          829 non-null    object  
 4   StartDate      829 non-null    object  
 5   EndDate        829 non-null    object  
 6   AreaHa         829 non-null    float64 
 7   PerimeterM     829 non-null    float64 
 8   Verdate        829 non-null    object  
 9   Type           829 non-null    object  
 10  Season         829 non-null    object  
 11  ID             829 non-null    int64   
 12  Year           829 non-null    int64   
 13  Year_Con       829 non-null    object  
 14  Shape__Area    829 non-null    float64 
 15  Shape__Length  829 non-null    float64 
 16  geometry       829 non-null    geometry
 17  index_right    829 non-nul

In [51]:
nsw_fire_new.to_file("./dataset/nsw_2019_sa2")

C:\Users\n9502271\AppData\Local\Temp\ipykernel_36728\2391650826.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nsw_fire_new.to_file("./dataset/nsw_2019_sa2")
